In [1]:
import time

class PID:
    def __init__(self, Kp=1.0, Ki=0.0, Kd=0.0, setpoint=0.0):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.setpoint = setpoint
        self._prev_time = time.time()
        self._prev_error = 0.0
        self._integral = 0.0

    def update(self, measured_value):
        current_time = time.time()
        dt = current_time - self._prev_time
        self._prev_time = current_time

        error = self.setpoint - measured_value
        self._integral += error * dt
        derivative = (error - self._prev_error) / dt if dt > 0 else 0.0

        output = self.Kp * error + self.Ki * self._integral + self.Kd * derivative
        self._prev_error = error
        return output

    def set_Kp(self, Kp): self.Kp = Kp
    def set_Ki(self, Ki): self.Ki = Ki
    def set_Kd(self, Kd): self.Kd = Kd
    def set_setpoint(self, setpoint): self.setpoint = setpoint


In [2]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

model.load_state_dict(torch.load('best_steering_model_xy.pth'))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

import torchvision.transforms as transforms
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std  = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]


In [3]:
# --- JetBot model-based line following (unified + error display) ---
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg, Robot
import cv2, numpy as np, time
import torch, torchvision
import torchvision.transforms as transforms
import PIL.Image

# --------------------------------------------------------------------
# Model assumptions
# --------------------------------------------------------------------
assert 'model' in globals(), "Run the model-loading cell first (defines model)."
device = next(model.parameters()).device

# --- Normalization (match training & overlay) ---
mean = torch.tensor([0.485, 0.456, 0.406], device=device).half()
std = torch.tensor([0.229, 0.224, 0.225], device=device).half()
IN_W, IN_H = 224, 224  # Model input size

def preprocess_frame_bgr(image_bgr):
    rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    rgb = cv2.resize(rgb, (IN_W, IN_H), interpolation=cv2.INTER_AREA)
    t = transforms.functional.to_tensor(PIL.Image.fromarray(rgb)).to(device).half()
    t.sub_(mean[:, None, None]).div_(std[:, None, None])
    return t[None, ...]

def xy_to_px(x, y, W, H):
    px = int((x * 0.5 + 0.5) * (W - 1))
    py = int((y * 0.5 + 0.5) * (H - 1))
    return max(0, min(W-1, px)), max(0, min(H-1, py))

# --- Simple PID ---
class PID:
    def __init__(self, Kp=0.6, Ki=0.0, Kd=0.05, setpoint=0.0):
        self.Kp, self.Ki, self.Kd = Kp, Ki, Kd
        self.setpoint = setpoint
        self._prev_err = 0.0
        self._prev_t = None
        self._integ = 0.0
    
    def set_Kp(self, v):
        self.Kp = float(v)
    
    def set_Kd(self, v):
        self.Kd = float(v)
    
    def set_setpoint(self, v):
        self.setpoint = float(v)
    
    def reset(self):
        self._prev_err = 0.0
        self._prev_t = None
        self._integ = 0.0
    
    def update(self, measurement, now=None):
        err = (self.setpoint - 0.0) - measurement  # setpoint=0 => err=-measurement
        t = time.time() if now is None else now
        if self._prev_t is None:
            dt = 0.0
            d_err = 0.0
        else:
            dt = max(1e-3, t - self._prev_t)
            d_err = (err - self._prev_err) / dt
        self._integ += err * dt
        out = self.Kp * err + self.Ki * self._integ + self.Kd * d_err
        self._prev_err = err
        self._prev_t = t
        return out, err  # return both control and raw error

# -------------------------
# Parameters
# -------------------------
forward_speed_default = 0.12
steering_bias_default = 0.00

camera = Camera.instance(width=IN_W, height=IN_H)
robot = Robot()

# -------------------------
# UI widgets
# -------------------------
view_widget = widgets.Image(format='jpeg', width=IN_W, height=IN_H)
start_button = widgets.Button(description="START", button_style='success',
                               layout=widgets.Layout(width='120px', height='40px'))
stop_button = widgets.Button(description="STOP", button_style='danger',
                              layout=widgets.Layout(width='120px', height='40px'))

kp_slider = widgets.FloatSlider(value=0.60, min=0.0, max=2.0, step=0.001, description='Kp')
kd_slider = widgets.FloatSlider(value=0.05, min=0.0, max=0.5, step=0.001, description='Kd')
speed_slider = widgets.FloatSlider(value=forward_speed_default, min=0.0, max=0.6, step=0.005, description='Speed')
bias_slider = widgets.FloatSlider(value=steering_bias_default, min=-0.5, max=0.5, step=0.005, description='Bias')

x_readout = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.001, description='x', readout=True)
y_readout = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.001, description='y', readout=True)
ang_readout = widgets.FloatSlider(value=0.0, min=-180.0, max=180.0, step=0.1, description='angle (deg)', readout=True)

display(
    widgets.VBox([
        view_widget,
        widgets.HBox([start_button, stop_button]),
        widgets.HBox([kp_slider, kd_slider]),
        widgets.HBox([speed_slider, bias_slider]),
        widgets.HBox([x_readout, y_readout, ang_readout]),
    ])
)

# -------------------------
# PID setup
# -------------------------
steer_pid = PID(Kp=kp_slider.value, Ki=0.0, Kd=kd_slider.value, setpoint=0.0)

def reset_pid(*_):
    steer_pid.reset()
    steer_pid.set_Kp(kp_slider.value)
    steer_pid.set_Kd(kd_slider.value)
    steer_pid.set_setpoint(0.0)

kp_slider.observe(reset_pid, names='value')
kd_slider.observe(reset_pid, names='value')

# -------------------------
# Control state
# -------------------------
running = False

def on_start(_):
    global running
    running = True

def on_stop(_):
    global running
    running = False
    robot.stop()

start_button.on_click(on_start)
stop_button.on_click(on_stop)

def clamp(v, lo, hi):
    return max(lo, min(hi, float(v)))

# -------------------------
# Main callback (now with live error display)
# -------------------------
@torch.no_grad()
def follow_model_xy(change):
    global running, steer_pid
    frame_bgr = change['new']
    if frame_bgr is None:
        return
    
    vis = frame_bgr.copy()
    H, W = vis.shape[:2]
    
    xy = model(preprocess_frame_bgr(frame_bgr)).float().squeeze().tolist()
    x, y = float(xy[0]), float(xy[1])
    
    x_readout.value = x
    y_readout.value = y
    ang_readout.value = float(np.clip(x * 90.0, -180.0, 180.0))
    
    steer_pid.set_Kp(kp_slider.value)
    steer_pid.set_Kd(kd_slider.value)
    steer_pid.set_setpoint(0.0)
    
    # Update PID and get error
    u, error = steer_pid.update(x)
    u += bias_slider.value
    u = clamp(u, -1.0, 1.0)
    
    fwd = speed_slider.value
    left, right = clamp(fwd - u, 0.0, 1.0), clamp(fwd + u, 0.0, 1.0)
    
    # Draw overlays
    px, py = xy_to_px(x, y, W, H)
    cv2.circle(vis, (px, py), 5, (0, 0, 255), -1)
    cv2.putText(vis, f"Kp={kp_slider.value:.2f} Kd={kd_slider.value:.3f} u={u:.3f}",
                (8, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (40,255,40), 1, cv2.LINE_AA)
    cv2.putText(vis, f"Error={error:.3f}",
                (8, H - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2, cv2.LINE_AA)
    
    if running:
        robot.set_motors(left, right)
    else:
        robot.stop()
    
    view_widget.value = bgr8_to_jpeg(vis)

# Attach callback
camera.observe(follow_model_xy, names='value')

# Optional cleanup:
# camera.unobserve(follow_model_xy, names='value'); robot.stop(); camera.stop()

In [10]:
robot.stop()

In [ ]:
camera.stop()